# Two Pointers: Debug-First Interview Notebook

This notebook is a markdown-heavy guide with detailed debugging steps and runnable trace code.

Covered problems (5):
1. Valid Palindrome
2. Two Sum II (Input Array Is Sorted)
3. Container With Most Water
4. 3Sum
5. Remove Duplicates from Sorted Array

## Two-Pointer Pattern Guide

### When to use it
1. Need relationships between elements from two sides or in a sorted range.
2. Brute force repeats comparisons that can be skipped by monotonic moves.
3. You want O(1) extra memory where possible.

### Core variants
1. Opposite-end pointers (left/right)
2. Same-direction pointers (slow/fast)
3. Fixed index + two pointers (3Sum style)

### Debug workflow for any two-pointer problem
1. Write movement rules before coding.
2. Trace one small example by hand and mark each pointer move.
3. Log every move with reason: too small / too large / matched / skip duplicate.
4. Confirm loop invariant stays true after every move.
5. Verify termination: at least one pointer must move each loop.

## Problem 1: Valid Palindrome (LeetCode 125)

### Statement
Given a string `s`, return `True` if it is a palindrome after removing non-alphanumeric characters and ignoring case.

### Debugging steps
1. Start `left=0`, `right=len(s)-1`.
2. Skip invalid chars from left; log each skip.
3. Skip invalid chars from right; log each skip.
4. Compare normalized chars; on mismatch return `False`.
5. Move both inward and continue until crossing.

### Invariant
All checked mirrored pairs outside `[left, right]` already match under normalization.

In [ ]:
def valid_palindrome_debug(s: str, verbose: bool = True) -> bool:
    left, right = 0, len(s) - 1
    step = 1

    while left < right:
        while left < right and not s[left].isalnum():
            if verbose:
                print(f'step {step}: skip left index {left} char={s[left]!r}')
            left += 1
            step += 1

        while left < right and not s[right].isalnum():
            if verbose:
                print(f'step {step}: skip right index {right} char={s[right]!r}')
            right -= 1
            step += 1

        left_char = s[left].lower()
        right_char = s[right].lower()

        if verbose:
            print(f'step {step}: compare left={left}({left_char!r}) vs right={right}({right_char!r})')

        if left_char != right_char:
            if verbose:
                print('mismatch -> return False')
            return False

        left += 1
        right -= 1
        step += 1

    if verbose:
        print('pointers crossed -> return True')
    return True


In [ ]:
valid_palindrome_debug('A man, a plan, a canal: Panama')

## Problem 2: Two Sum II (LeetCode 167)

### Statement
Given a sorted array `numbers` and `target`, return 1-indexed positions of two values that sum to target.

### Debugging steps
1. Set `left=0`, `right=n-1`.
2. Compute `current = numbers[left] + numbers[right]`.
3. If `current < target`, move left (need larger sum).
4. If `current > target`, move right (need smaller sum).
5. If equal, return immediately.

### Invariant
Search space is always exactly the window `[left, right]`; eliminated pairs cannot satisfy target.

In [ ]:
def two_sum_ii_debug(numbers: list[int], target: int, verbose: bool = True) -> list[int]:
    left, right = 0, len(numbers) - 1
    step = 1

    while left < right:
        current = numbers[left] + numbers[right]
        if verbose:
            print(
                f'step {step}: left={left}({numbers[left]}), right={right}({numbers[right]}), sum={current}'
            )

        if current == target:
            if verbose:
                print('found target -> return 1-indexed pair')
            return [left + 1, right + 1]
        if current < target:
            if verbose:
                print('sum too small -> move left')
            left += 1
        else:
            if verbose:
                print('sum too large -> move right')
            right -= 1

        step += 1

    if verbose:
        print('no solution in window -> return empty')
    return []


In [ ]:
two_sum_ii_debug([2, 7, 11, 15], 9)

## Problem 3: Container With Most Water (LeetCode 11)

### Statement
Choose two lines so container area is maximized: `area = min(height[left], height[right]) * (right-left)`.

### Debugging steps
1. Evaluate area for current endpoints.
2. Update best if larger.
3. Move pointer with shorter height (only move that may improve limiting height).
4. Repeat until pointers meet.

### Invariant
Any area using the shorter boundary with a smaller width cannot beat current choice if shorter side stays unchanged.

In [ ]:
def container_with_most_water_debug(height: list[int], verbose: bool = True) -> int:
    left, right = 0, len(height) - 1
    best = 0
    step = 1

    while left < right:
        width = right - left
        h = min(height[left], height[right])
        area = h * width
        best = max(best, area)

        if verbose:
            print(
                f'step {step}: left={left}({height[left]}), right={right}({height[right]}), '
                f'width={width}, area={area}, best={best}'
            )

        if height[left] <= height[right]:
            if verbose:
                print('move left (shorter or equal side)')
            left += 1
        else:
            if verbose:
                print('move right (shorter side)')
            right -= 1

        step += 1

    if verbose:
        print(f'finished -> best area={best}')
    return best


In [ ]:
container_with_most_water_debug([1, 8, 6, 2, 5, 4, 8, 3, 7])

## Problem 4: 3Sum (LeetCode 15)

### Statement
Return all unique triplets `[a, b, c]` such that `a + b + c == 0`.

### Debugging steps
1. Sort input.
2. Fix anchor `i`; skip duplicate anchors.
3. Use `left/right` on suffix.
4. If sum too small, move left; too large, move right.
5. On zero, record triplet and skip duplicate neighbors.

### Invariant
For each anchor, `[left, right]` is the remaining pair search window; sorted order makes pointer movement monotonic and safe.

In [ ]:
def three_sum_debug(nums: list[int], verbose: bool = True) -> list[list[int]]:
    nums = sorted(nums)
    results: list[list[int]] = []

    if verbose:
        print(f'sorted nums = {nums}')

    for i in range(len(nums) - 2):
        if i > 0 and nums[i] == nums[i - 1]:
            if verbose:
                print(f'skip duplicate anchor i={i} value={nums[i]}')
            continue

        left, right = i + 1, len(nums) - 1
        if verbose:
            print(f'anchor i={i}, value={nums[i]}, left={left}, right={right}')

        while left < right:
            total = nums[i] + nums[left] + nums[right]
            if verbose:
                print(
                    f'  check ({nums[i]}, {nums[left]}, {nums[right]}) -> total={total}'
                )

            if total == 0:
                results.append([nums[i], nums[left], nums[right]])
                if verbose:
                    print(f'  found {results[-1]}')
                left += 1
                right -= 1

                while left < right and nums[left] == nums[left - 1]:
                    if verbose:
                        print(f'  skip duplicate left at {left} value={nums[left]}')
                    left += 1
                while left < right and nums[right] == nums[right + 1]:
                    if verbose:
                        print(f'  skip duplicate right at {right} value={nums[right]}')
                    right -= 1
            elif total < 0:
                if verbose:
                    print('  total too small -> move left')
                left += 1
            else:
                if verbose:
                    print('  total too large -> move right')
                right -= 1

    if verbose:
        print(f'final triplets = {results}')
    return results


In [ ]:
three_sum_debug([-1, 0, 1, 2, -1, -4])

## Problem 5: Remove Duplicates from Sorted Array (LeetCode 26)

### Statement
Given sorted `nums`, remove duplicates in-place and return `k`, the count of unique elements in the prefix.

### Debugging steps
1. `slow` points to last unique position.
2. `fast` scans from left to right.
3. If `nums[fast] != nums[slow]`, write new unique value at `slow+1` and move `slow`.
4. Continue until `fast` reaches end.

### Invariant
At every step, `nums[:slow+1]` holds unique sorted values seen so far.

In [ ]:
def remove_duplicates_debug(nums: list[int], verbose: bool = True) -> tuple[int, list[int]]:
    if not nums:
        if verbose:
            print('empty list -> k=0')
        return 0, []

    slow = 0
    step = 1

    for fast in range(1, len(nums)):
        if verbose:
            print(
                f'step {step}: slow={slow}({nums[slow]}), fast={fast}({nums[fast]})'
            )

        if nums[fast] != nums[slow]:
            slow += 1
            nums[slow] = nums[fast]
            if verbose:
                print(f'  new unique -> write nums[{slow}]={nums[fast]}')
        else:
            if verbose:
                print('  duplicate -> skip')

        step += 1

    k = slow + 1
    if verbose:
        print(f'finished -> k={k}, unique prefix={nums[:k]}')
    return k, nums[:k]


In [ ]:
remove_duplicates_debug([0, 0, 1, 1, 1, 2, 2, 3, 3, 4])

## Debugging Checklist (Use for all 5 problems)

1. Confirm pointer initialization values.
2. Ensure at least one pointer moves every loop iteration.
3. Log exact reason for each move (small/large/match/duplicate/invalid-char).
4. Re-check loop condition (`left < right`, `fast < n`, etc.).
5. Validate edge cases: empty input, one element, all duplicates, all invalid chars.

Run each code cell with `verbose=True` first, then turn verbose off after you trust the logic.